In [24]:
# get city list 
import geonamescache

# get lat lon for cities
from geopy.geocoders import Nominatim

# iterate into data frame
from operator import itemgetter
import pandas as pd


# load json in to dict
import json
# send reqiest for api
import requests

# create a list of date 
from datetime import datetime, timedelta

import plotly.express as px


#### get cities name and population

In [10]:
# Create an object from GeonamesCache
gc = geonamescache.GeonamesCache()
geolocator = Nominatim(user_agent="geoapiExercises")

# Get a list of all cities
all_cities = gc.get_cities()

# Extract cities in the South East of England
england_cities = [city for city in all_cities.values() if city['countrycode'] == 'GB' and city['admin1code'] == 'ENG' and city['population'] is not None]

# Sort the cities by population in descending order and select the top 5
top_5_cities = sorted(england_cities, key=itemgetter('population'), reverse=True)[:5]

# Add latitude and longitude to the selected cities
for city in top_5_cities:
    location = geolocator.geocode(city['name'])
    if location:
        city['latitude'] = location.latitude
        city['longitude'] = location.longitude

# Create a DataFrame from the selected cities
df = pd.DataFrame(top_5_cities, columns=['name', 'population', 'latitude', 'longitude'])

# Print the DataFrame
print(df)

         name  population   latitude  longitude
0      London     8961989  51.507446  -0.127765
1  Birmingham     1144919  52.479699  -1.902691
2   Liverpool      864122  53.407199  -2.991680
3   Sheffield      556500  53.380663  -1.470228
4       Leeds      516298  53.797418  -1.543794


         name  population   latitude  longitude
0      London     8961989  51.507446  -0.127765
1  Birmingham     1144919  52.479699  -1.902691
2   Liverpool      864122  53.407199  -2.991680
3   Sheffield      556500  53.380663  -1.470228
4       Leeds      516298  53.797418  -1.543794

#### iterate  api  

In [13]:
# create a empty list
urls_list=[]
# define start date 
start_date = datetime(2023, 12, 1)
# define end date 
end_date = datetime.now()
# generate list of date from 2023-12-01
date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
# define apikey
apiKey="638f8ce32f8241bcb4e15254231911"
# iterate dataframe to get index and row then replace the parameter and add to the defined list 
for index, row in df.iterrows():
    for date in date_list:
        urls_list.append(f"http://api.weatherapi.com/v1/history.json?key={apiKey}&q={row['name']}&dt={date.strftime('%Y-%m-%d')}")

#### get a data from api 

In [14]:
# create an empty list
weather_data = []
# iterate urls_list created in code 
for url in urls_list:
    # send request to api and get response in to response variable 
    response = requests.get(url)
    # load the response.text into json_result variable
    json_result = json.loads(response.text)
    # because there is 24 hours per each days its needed to go through 
    # each of them and extract from jason document as key, value
    for i in range(len(json_result['forecast']['forecastday'][0]['hour'])):
        hourly_data = json_result['forecast']['forecastday'][0]['hour'][i]
        row={
            'date_time' : hourly_data['time'],
            'country' : json_result['location']['country'],
            'city' : json_result['location']['name'],
            'region' : json_result['location']['region'],
            'chance_of_rain' : hourly_data['chance_of_rain'],
            'chance_of_snow' : hourly_data['chance_of_snow'],
            'cloud_cover'    : hourly_data['cloud'],
            'humidity' : hourly_data['humidity'],
            'visibility_km' : hourly_data['vis_km'],
            'wind_direction' : hourly_data['wind_dir'],
            'wind_speed_kph' : hourly_data['wind_kph'],
            'weather_condition' : hourly_data['condition']['text'],
            'avghumidity' : json_result['forecast']['forecastday'][0]['day']['avghumidity'],
            'avgtemp_c' : json_result['forecast']['forecastday'][0]['day']['avgtemp_c'],
            'avgtemp_f' : json_result['forecast']['forecastday'][0]['day']['avgtemp_f'],
            'avgvis_km' : json_result['forecast']['forecastday'][0]['day']['avgvis_km'],
            'avgvis_miles' : json_result['forecast']['forecastday'][0]['day']['avgvis_miles'],
            'maxtemp_c': json_result['forecast']['forecastday'][0]['day']['maxtemp_c'],
            'maxtemp_f' : json_result['forecast']['forecastday'][0]['day']['maxtemp_f'],
            'maxwind_kph' : json_result['forecast']['forecastday'][0]['day']['maxwind_kph'],
            'maxwind_mph' : json_result['forecast']['forecastday'][0]['day']['maxwind_mph'],
            'mintemp_c' : json_result['forecast']['forecastday'][0]['day']['mintemp_c'],
            'mintemp_f' : json_result['forecast']['forecastday'][0]['day']['mintemp_f'],
            'mintemp_f' : json_result['forecast']['forecastday'][0]['day']['mintemp_f']
        }
        # add all the extracted info into a list 
        weather_data.append(row)

# Create a DataFrame from the list of extracted data
weather_df = pd.DataFrame(weather_data)

# Display the dataframe
weather_df.head()

,date_time,country,city,region,chance_of_rain,chance_of_snow,cloud_cover,humidity,visibility_km,wind_direction,...,avgtemp_c,avgtemp_f,avgvis_km,avgvis_miles,maxtemp_c,maxtemp_f,maxwind_kph,maxwind_mph,mintemp_c,mintemp_f
0,2023-12-01 00:00,United Kingdom,London,"City of London, Greater London",0,0,3,71,10.0,N,...,1.8,35.2,9.3,5.0,4.4,39.8,18.4,11.4,0.6,33.0
1,2023-12-01 01:00,United Kingdom,London,"City of London, Greater London",0,0,4,72,10.0,N,...,1.8,35.2,9.3,5.0,4.4,39.8,18.4,11.4,0.6,33.0
2,2023-12-01 02:00,United Kingdom,London,"City of London, Greater London",0,0,6,73,10.0,N,...,1.8,35.2,9.3,5.0,4.4,39.8,18.4,11.4,0.6,33.0
3,2023-12-01 03:00,United Kingdom,London,"City of London, Greater London",0,0,9,78,10.0,N,...,1.8,35.2,9.3,5.0,4.4,39.8,18.4,11.4,0.6,33.0
4,2023-12-01 04:00,United Kingdom,London,"City of London, Greater London",0,0,7,78,10.0,N,...,1.8,35.2,9.3,5.0,4.4,39.8,18.4,11.4,0.6,33.0


In [15]:
file_path_name=r"E:\Class_PrincipleOfDataIntelligencePrograming\weather_df.csv"
weather_df.to_csv(file_path_name, encoding= 'utf-16',sep="\t")

In [12]:
# Send a GET request to the API
url = 'http://api.weatherapi.com/v1/history.json?key=638f8ce32f8241bcb4e15254231911&q=London&dt=2023-12-01'
response = requests.get(url)

# Store the JSON result in a variable
json_result = json.loads(response.text)

weather_data = []

for i in range(len(json_result['forecast']['forecastday'][0]['hour'])):
    hourly_data = json_result['forecast']['forecastday'][0]['hour'][i]
    row={
        'date_time' : hourly_data['time'],
        'country' : json_result['location']['country'],
        'city' : json_result['location']['name'],
        'region' : json_result['location']['region'],
        'chance_of_rain' : hourly_data['chance_of_rain'],
        'chance_of_snow' : hourly_data['chance_of_snow'],
        'cloud_cover'    : hourly_data['cloud'],
        'humidity' : hourly_data['humidity'],
        'visibility_km' : hourly_data['vis_km'],
        'wind_direction' : hourly_data['wind_dir'],
        'wind_speed_kph' : hourly_data['wind_kph'],
        'weather_condition' : hourly_data['condition']['text'],
        'avghumidity' : json_result['forecast']['forecastday'][0]['day']['avghumidity'],
        'avgtemp_c' : json_result['forecast']['forecastday'][0]['day']['avgtemp_c'],
        'avgtemp_f' : json_result['forecast']['forecastday'][0]['day']['avgtemp_f'],
        'avgvis_km' : json_result['forecast']['forecastday'][0]['day']['avgvis_km'],
        'avgvis_miles' : json_result['forecast']['forecastday'][0]['day']['avgvis_miles'],
        'maxtemp_c': json_result['forecast']['forecastday'][0]['day']['maxtemp_c'],
        'maxtemp_f' : json_result['forecast']['forecastday'][0]['day']['maxtemp_f'],
        'maxwind_kph' : json_result['forecast']['forecastday'][0]['day']['maxwind_kph'],
        'maxwind_mph' : json_result['forecast']['forecastday'][0]['day']['maxwind_mph'],
        'mintemp_c' : json_result['forecast']['forecastday'][0]['day']['mintemp_c'],
        'mintemp_f' : json_result['forecast']['forecastday'][0]['day']['mintemp_f'],
        'mintemp_f' : json_result['forecast']['forecastday'][0]['day']['mintemp_f']
    }
    weather_data.append(row)

# Create a DataFrame from the list of extracted data
weather_df = pd.DataFrame(weather_data)

# Display the dataframe
weather_df.head()

,date_time,country,city,region,chance_of_rain,chance_of_snow,cloud_cover,humidity,visibility_km,wind_direction,...,avgtemp_c,avgtemp_f,avgvis_km,avgvis_miles,maxtemp_c,maxtemp_f,maxwind_kph,maxwind_mph,mintemp_c,mintemp_f
0,2023-12-01 00:00,United Kingdom,London,"City of London, Greater London",0,0,3,71,10.0,N,...,1.8,35.2,9.3,5.0,4.4,39.8,18.4,11.4,0.6,33.0
1,2023-12-01 01:00,United Kingdom,London,"City of London, Greater London",0,0,4,72,10.0,N,...,1.8,35.2,9.3,5.0,4.4,39.8,18.4,11.4,0.6,33.0
2,2023-12-01 02:00,United Kingdom,London,"City of London, Greater London",0,0,6,73,10.0,N,...,1.8,35.2,9.3,5.0,4.4,39.8,18.4,11.4,0.6,33.0
3,2023-12-01 03:00,United Kingdom,London,"City of London, Greater London",0,0,9,78,10.0,N,...,1.8,35.2,9.3,5.0,4.4,39.8,18.4,11.4,0.6,33.0
4,2023-12-01 04:00,United Kingdom,London,"City of London, Greater London",0,0,7,78,10.0,N,...,1.8,35.2,9.3,5.0,4.4,39.8,18.4,11.4,0.6,33.0


##### plot1 average temperature of all dataset in days  

In [36]:

fig = px.bar(weather_df, x='city', y='avgtemp_c', text='avgtemp_c', title='Average Temperature in Each City')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(title_text='Average Temperature in Each City', xaxis_title='City', yaxis_title='Average Temperature (°C)')
fig.update_layout(width=800, height=600)  # Change the figure size
fig.update_yaxes(type='linear')  # Ensure y-axis type is linear
fig.update_xaxes(categoryorder='total descending')  # Sort the bars in ascending order
fig.show()

In [44]:
fig = px.line(weather_df, x='date_time', y='humidity', color='city', title='humidity for Each City Over 24 Hours')
fig.update_layout(width=1200, height=600)  # Change the figure size
fig.show()

KeyError: "None of [Index(['chance_of_rain', 'chance_of_snow', 'cloud_cover', 'humidity',\n       'visibility_km', 'wind_speed_kph', 'avghumidity', 'avgtemp_c',\n       'avgtemp_f', 'avgvis_km', 'avgvis_miles', 'maxtemp_c', 'maxtemp_f',\n       'maxwind_kph', 'maxwind_mph', 'mintemp_c', 'mintemp_f'],\n      dtype='object')] are in the [columns]"